# Data Analysis Tutorial

## Prerequisites
You’ll need to know a bit of Python. For a refresher, see the [Python tutorial](https://docs.python.org/3/tutorial/)

### Osiris Version
The version of Osiris is slightly different to Michael's version. This version contains the realigner built into the fReader. 
Hence please use the version of Osiris I have provided. For versions compatible to the original Osiris, consult the labnotes.

### Python Version
Python 3.x (Ensure you have a compatible version of Python 3.x installed. Python 2.x is not supported.)


### Required Libraries and Modules
The following libraries and modules are required to run the provided code:



In [4]:
!pip install numpy hist matplotlib mplhep pandas plotly scipy tqdm

In [61]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"//data//"+p, data_list)) # insert your file

### Tutorial Coverage

This is a quick start tutorial example. In this tutorial you will find

1. Example usage of various functions, with high level explainations
2. How data gathered from rawfileReader, how data are processed with each other
3. Reproducing all essential results obtained so far
4. Comments on limitations and warning for usage
5. Tutorial on Timing Analysis, Reconstruction and Realigner

Results including alignment metric, TDC status metric, Reconstruction, time of flight analysis, and how to integrate these systems together

<span style="color:green">For a documentation styled explaination for each function, see other notebook</span>.

### Learning Objectives
After this tutorial, you should be able to understand how the data are extracted from the Raw file, how these data are then used in each function to do their designated tasks.

Understand the data structure, and how to use these data outputs to plot certain plots.

Using class functions and decorators to extract internal variables (Or use PyCharm...)

### Used Data
All example data used are latest data proAnubis_240716_2142
The Reconstruction algorithm was adapted to the latest data behavior where noise bursts were seen, so for the sake of speed, noise bursts were ignored.

### Usage Warning
The code was adapted to analyse data in any situations, so the data structures were written prioritising mutability and easy extraction. This also means that for certain tasks, you might need to adjust low level functions' behavior. PyCharm does this much better than VsCode...

# The Basics

The rawFileReader.py is used to read the content of .raw files that stores the data from runs of proAnubis setup. .raw files contains uncompressed binary datas that requires the rawFileReader to decode and turn into useful analysis. 

The Pro_Anubis is run using triggers, which triggers data taking for 1250ns if there are 4 eta side strips triggered within a fixed time frame. Each trigger is called an <span style="color:cyan">event</span>. 

An event contains a <span style="color:cyan">header</span>, which details the content of the event, then the events are written in binary <span style="color:cyan">words</span>, and n <span style="color:cyan">End of block</span> signals a termination of an <span style="color:cyan">event</span>

The <span style="color:cyan">event</span> are written as `proAnubEvent` object, which contains `tdcEvent` objects. 


### tdcEvent Class

#### Overview
The `tdcEvent` class is what the output from rawfileReaders are

#### Class Variables

##### `header`
- **Type:** `Any`
- **Description:** Represents the header word of the event. 

##### `words`
- **Type:** `List[Any]`
- **Default:** `[]`
- **Description:** A list containing words for each event. These words represent the hits.

##### `time`
- **Type:** `Any`
- **Description:** Holds the timing information for the event, a time stamp in Daytime format.

##### `EOB`
- **Type:** `Any`
- **Default:** `None`
- **Description:** Represents the end of block indicator. This is used to signify the end of a data block within a trigger.

##### `qual`
- **Type:** `int`
- **Default:** `0`
- **Description:** A quality check indicator. The `qual` value provides information on data integrity:
  - If `qual == 0`, it means the data has not experienced corruption that required corrections in the raw file reader.
  - If `qual != 0`, it indicates that data corruption occurred and was captured by the system.

#### Constructor

The `__init__` method initializes the `tdcEvent` class with the provided parameters:

def __init__(self, header, time, words = [], eob=None, qual=0):
    self.header = header
    self.words = words
    self.time = time
    self.EOB = eob
    self.qual = qual


# Data Extraction

To extract the data from raw files, the `get_aligned_events` function is used. At default, it will output globally aligned events untill the end of the file. Usually the file is very large, hene ann event loop is sggested to end early

In [5]:
importlib.reload(rawFileReader) # Reload fReader
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # reload in the classs object
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        #print(event_chunk[0].tdcEvents[0].time)
        pbar.update(1)
    

Processing Events:   0%|          | 0/100 [00:00<?, ?Events/s]

Processing Events: 100%|██████████| 100/100 [00:02<00:00, 39.84Events/s]


## Internal variable extraction

some internal variables might be interesting and worth studying. There are a variety of ways to extract them, the simplest way is to go back to the code, and add an extra class variable and record all the information in an instance. This however, is unsustainable and the usual gist is that you shouldn't rewrite a working system just to extract some variables you need once...

### Decorators
Decorators are a great tool to extract and capture information within the event run time without altering the original code. please read [Decorators tutorial](https://realpython.com/primer-on-python-decorators/) for more information
Please note you need to restart the environment if the function rewrote by the decorator runs into issue, because the function will not return to original state

An example decorator for printing out the alignment insertions done without altering the original function in fReader is printed here

In [90]:
#Example Decorators to capture things in the loop so you don't need to redo any calculations later......
def debug_decorator(processedEvents):
    def inner_decorator(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            insertion_list = result
            update = args[0]
            if insertion_list != [0,0,0,0,0]:
                print(f'New alignment, Event chunk {processedEvents}, insertions {insertion_list}, Updates: {update}')
            return result
        return wrapper
    return inner_decorator

In [91]:
#Jupyter notebook doesn't reload your import even when the content of the file is changed. This is crucial
importlib.reload(rawFileReader)
#The interval is the amount of chunks you want to do realignment analysis with. This also determines the size of the chunk your output is.
interval = 100
fReader = rawFileReader.fileReader(file_path[0])
# Order is the alignment comparisons you need to specify. for each sublist in order is a pairwise comparison between 2 TDCs.
order = [(0,1), (1,2), (2,3), (3,4)] #Your order to the TDC alignment
# max_process_event_chunk is used to terminate early in the file reading loop. Your total number of events read will be defined as interval * max_proess_event_chunk
max_process_event_chunk = 1000
# some initialisation to store things
processedEvents = 0
events = []

#Remember the original function
original_ConstructEventInsertionList = ATools.ConstructEventInsertionList 
# condition to end event loop early
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    #Apply your decorator to change the function
    ATools.ConstructEventInsertionList  = debug_decorator(processedEvents)(original_ConstructEventInsertionList)
    event = fReader.get_aligned_events(order=order, interval=interval)
#reset afterwards
ATools.ConstructEventInsertionList  = original_ConstructEventInsertionList


New alignment, Event chunk 1, insertions [0, 1, 4, 5, 7], Updates: [-1, -3, -1, -2]
New alignment, Event chunk 387, insertions [1, 1, 0, 1, 1], Updates: [0, 1, -1, 0]
New alignment, Event chunk 399, insertions [0, 1, 1, 0, 0], Updates: [-1, 0, 1, 0]
New alignment, Event chunk 424, insertions [0, 1, 1, 0, 0], Updates: [-1, 0, 1, 0]
New alignment, Event chunk 434, insertions [0, 1, 1, 0, 0], Updates: [-1, 0, 1, 0]
New alignment, Event chunk 649, insertions [1, 1, 0, 1, 1], Updates: [0, 1, -1, 0]
New alignment, Event chunk 826, insertions [1, 1, 1, 0, 0], Updates: [0, 0, 1, 0]
New alignment, Event chunk 827, insertions [0, 1, 1, 1, 1], Updates: [-1, 0, 0, 0]


### Re_calculation

Athough some calculations are processed internally, it might be easier to reprocess the output event list again by redoing the necessary calculations. This will result in redundant calculations, and not possible for all calculations, such as TDC monitoring metric and realignment words used. 

Both fReader and this code is written to ensure the data is calculated in chunks, this avoids memory overflow, and allow the program to start and terminate anywhere one desires.

When the event chunk is loaded in to the instance, calculation has to be done within this instance, and saved into an external variable. Below is an example of using the function RTools.`find_tdc_event_count` to calculate the number of headers in each TDC and writing into a buffer `tdc_event_count_buffer` for that instance, then the information from the buffer is tranfered into an external memory storage `tdc_event_count` outside the process loop

Similarly, the alignment metric can be calculated from ATools.`calcAvgAlign` (for more information refer to documentation).

In [91]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[2]) # reload in the classs object
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 1000 # End the loop early
processedEvents = 0 # Initialisation

#Initialise variables to store the results
mets = [[] for tdc in range(5)]
empty_header = 0 # Counting the number of empty headers
emtpy_events_with_header = [[], []] # prepare two lists of same dimensions for line graph plotting
tdc_event_count = [[] for tdc in range(5)]# prepare for histogram plotting


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
        tdc_event_count_buffer = RTools.find_tdc_event_count(event_chunk) # finding the number of headers
        [tdc_event_count[i].append(tdc_event_count_buffer[i]) for i in range(5)] # write from buffer to memory
        for idx, (i, j) in enumerate(order):
            x, y, l, m = ATools.find_tdc_alignment_metric(i, j) # determining which RPCs to use for aignment metric
            alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents) # determine the alignment metric
            empty_header += (alignMet == -1) # Calculating the number of empty headers
            emtpy_events_with_header[0].append(empty_header) # write to memory
            emtpy_events_with_header[1].append(processedEvents)
            mets[idx].append(alignMet) # write to memory
        pbar.update(1)
    

Processing Events: 100%|██████████| 1000/1000 [00:16<00:00, 60.99Events/s]


In [92]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))

for idx, item in enumerate(order):
    met = mets[idx]
    i, j = item
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')


ax.set_xlim(0, max_process_event_chunk)
ax.set_ylim(-1, 40)
ax.legend()
ax.set_title('Alignment graph')
ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
ax.set_xlabel('Processed Event chunks')
plt.show()

In [86]:
plt.figure(figsize=(10, 6))
plt.plot(emtpy_events_with_header[1], emtpy_events_with_header[0], marker='o')
plt.xlabel('Processed Events')
plt.ylabel('Only Header')
plt.title('Only Header vs Processed Events')
plt.grid(True)
plt.show()

In [83]:
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_event_count[tdc]
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{tdc}')

ax.set_xlim(0, max_process_event_chunk)
# ax.set_ylim(-1, 100)
ax.legend()
ax.set_title('TDC number of events recorded')
ax.set_ylabel('num of events')
ax.set_xlabel('Processed Event')
plt.show()
    

## Conclusion

fReader doesn't have many instances for storing metric, they are typically thrown out after use, hence it is very difficult to extract these out. However, it becomes easier as we move to Timing Analyser and Reconstructor where the class instances can be called directly to extract and record data. 

The general idea is, if the data is in Osiris, they are NOT stored so that it will run smoothly always. Other data like reconstructio and timing analyser will be stored in class instances, which will remain in memory as long you as you don't reload them

# The `Timing Analyser`

The `Timing_Analyser` class is designed to process and analyze timing data from event chunks. This class helps in calculating and visualizing time of flight analysis, residuals, and other related metrics for the pro_anubis detectors. It provides functionalities to update events, read TDC (Time-to-Digital Converter) time differences, calculate residuals, check eta trigger, and plot various data for analysis.

When using any class from `proAnubis_analysis_tools`, it is important that a new event loop format is used. The loop involves an initialisation, which clears all instances. When the event loop is entered, calculations are done through internal instances, meaning one must not TAnalyser, but to update the event_chunk through TAnalyser.`update_event`

One of the most useful metric to determine the corruption status is TAnalyser.`check_eta_trigger`. This function checks if each chunk has corruption and also count the total number of corrupted events with which RPC was involved in the corruption within the class instance

In [74]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            TAnalyser.update_event(event_chunk, processedEvents)
            status, failure = TAnalyser.check_eta_trigger()
            if not status:
                #print(f'Event chunk {processedEvents} failed the eta trigger check')
                #print(failure)
                pass
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:02<00:00, 51.42Events/s]


To plot these on a graph, note we are calling from TAnalyser to obtain the information we needed

In [75]:
event_counts_windows = [len(TAnalyser.count[count]) for count in range(7)]
total_windows = sum(event_counts_windows)
normalized_windows = [count / total_windows for count in event_counts_windows]
normalized_windows.append(normalized_windows[-1])
plt.figure(figsize=(12, 8))
r1 = list(range(7)) + [6.5]

# plt.step(r1, normalized_linux, color='mediumseagreen', linestyle='-', linewidth=2, markersize=6, label='0-15000', alpha=1, where='mid')
plt.step(r1, normalized_windows, color='dodgerblue', linestyle='-', linewidth=2, markersize=6, label='0-15000?', alpha=1, where='mid')

plt.xlabel('Trigger Number')
plt.ylabel('Number of Events Normalized')
plt.title('Normalized Event Count')
plt.ylim(0)
plt.xlim(0, 6.5)
plt.xticks(range(7))
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.legend()
plt.show()

As an example, one can also plot directly from class accessing the class variable. This makes no difference fundamentally, but it does make the code much clearer and immediately clear what you are plotting

In [13]:
TAnalyser.plot_rpc_involvement_histogram()

## TDC monitoring
The TDC monitoring script is innatly written in the fReader. Every 2500 events, the TDC status is monitored. This TDC status metric counts the number of events where the first time is within the "bad region" with an event time> 300 ns compare to the "good region" where the event time <300ns. It was typicaly found that the TDC is in an error state when the fraction is larger than 0.2

As a complementary output, TDC_info is also outputted to show the first hit time and first hit channels on each TDC.

In [108]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[2]) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 10000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
#Initialisation
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) 
        # get_aligned_events have a choice to output the tdc metric and tdc information by setting extract_tdc_mets to be True
        [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
        [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
        pbar.update(1)


Processing Events: 100%|██████████| 1000/1000 [00:02<00:00, 360.08Events/s]


In [109]:
import matplotlib.pyplot as plt
colors = ['blue', 'green', 'red', 'purple', 'orange']
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_mets[tdc]
    binsx = [x*250 for x in range(len(met))]
    ax.plot(binsx,met, label = f'tdc {tdc}', color = colors[tdc])

ax.set_xlim(0,max_process_event_chunk)
# ax.set_ylim(0,1)
ax.legend()
ax.set_title('TDC monitoring metric')
ax.set_ylabel('bad time behavior / nominal time behavior')
ax.set_xlabel('processed Event')
plt.show()

By plotting these out, one will see the first hit time and first hit channels vary between the good and bad regions

In [110]:
importlib.reload(TTools)
TTools.plot_tdc_error_times(Tot_TDC_info)
TTools.plot_tdc_error_times_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], output_pdf='TDC_first_hit_time.pdf')
TTools.plot_tdc_error_channels(Tot_TDC_info)
TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], tdcs_to_plot=None, output_pdf='TDC_first_hit_time_channel.pdf')

C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Timing_tools.py:305: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Getting the variables used to calculate the alignment metric is even more difficult, since the output data is not representative to what the code used to compute the alignment metric, hence we need decorators to capture the variable in question during the program's run time, record them, and plot them. Below is an example decorator designed to capture the event words used when doing realignments. 

Essentially it access the function in question, capturing its' input and output, and calculating the the min channels after the original calculation. 

You may ask why do I do this, well this is because the functino calculating this is step 4 deep in the code, you need to pretty much add this extra check in every functino it passes through, eventually extracting it to the top level. And you probably shouldn't mess with fReader that much..

In [83]:
class Capturer:
    def __init__(self):
        self.TDC_alignment_time = [[] for _ in range(5)]
        self.processedEvents = 0 

    def extra_calculation_decorator(self, func):
        def wrapper(*args, **kwargs):
            minTimes = [300, 300]
            minChans = [-1, -1]
            minRPC = [-1, -1]
            minWord = [-1, -1]
            tdc = [-1, -1]
            eta = [True, True]
            
            rpc1Hits = args[0]
            rpc2Hits = args[1]
            skipChans = kwargs.get('skipChans', [])

            result = func(*args, **kwargs)
            
            if result == -1:
                return result
            for hit in rpc1Hits:
                if hit.time < minTimes[0] and hit.channel not in skipChans:
                    minTimes[0] = hit.time
                    minChans[0] = hit.channel
                    minRPC[0] = hit.rpc
                    eta[0] = hit.eta
            for hit in rpc2Hits:
                if hit.time < minTimes[1] and hit.channel not in skipChans:
                    minTimes[1] = hit.time
                    minChans[1] = hit.channel
                    minRPC[1] = hit.rpc
                    eta[1] = hit.eta
            
            if -1 in minChans:
                return -1
            
            a = TTools.rpcHitToTdcChan(minRPC[0], minChans[0], eta[0])
            b = TTools.rpcHitToTdcChan(minRPC[1], minChans[1], eta[1])
            
            tdc[0], minWord[0] = a
            tdc[1], minWord[1] = b

            self.TDC_alignment_time[tdc[0]].append((minTimes[0], a, self.processedEvents))
            self.TDC_alignment_time[tdc[1]].append((minTimes[1], b, self.processedEvents))

            return result

        return wrapper

In [84]:
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(TTools)

Capturer = Capturer()

# Apply the decorator
original_testAlign = ATools.testAlign
ATools.testAlign = Capturer.extra_calculation_decorator(ATools.testAlign)

# Main loop
interval = 100  # Set your monitoring chunk size
fReader = rawFileReader.fileReader(file_path)  # Load in the class object
order = [[0, 1], [1, 2], [2, 3], [3, 4]]  # Order what you want to align
max_process_event_chunk = 200  # End the loop early
processedEvents = 0  # Initialization
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]

with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        Capturer.processedEvents = processedEvents
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        pbar.update(1)
# Return to original, or restart kernal
ATools.testAlign = original_testAlign



Processing Events: 100%|██████████| 200/200 [00:04<00:00, 42.90Events/s]


Hence you can find which channels and which timing are used for alignment metric calculation

In [63]:
importlib.reload(TTools)
TTools.plot_tdc_alignment_channels_custom_ranges(Capturer.TDC_alignment_time, [(0, 50), (50, 100)], output_pdf='Data_output/TDC_alignment_channels_used.pdf' )
TTools.plot_tdc_alignment_times_custom_ranges(Capturer.TDC_alignment_time, [(0, 50), (50, 100)], output_pdf='Data_output/TDC_alignment_time_used.pdf' )

NameError: name 'Capturer' is not defined

### time walk effect

It was found that signals takes time to travel from the FEB to the TDC, and on top of that, a systematic timing offset on each channel was also observed. Below is a tool written by Michael to read the timing difference between eta and phi channels, and averaging them across the entire chunk, and finally plotting them.

In [38]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    if event_chunk:
        TAnalyser.update_event(event_chunk, processedEvents)
        TAnalyser.readTDCTimeDiffs()
        
outDict = {'totDiffs':TAnalyser.totDiffs,
                    'nDiffs':TAnalyser.nDiffs,
                    'diffHists':TAnalyser.diffHists} 

The residual is calculated by applying the time walk correction, then averaging across the whole strip to a 2D plane. You should be able to see the time walk effect gone after applying the correction, and we are left with systematic corrections only

In [39]:
residEta, residPhi = TAnalyser.Calculate_Residual_and_plot_TDC_Time_Diffs( 
                                                     pdf_filename='Data_output/TDC_time_diffs.pdf', 
                                                     max_itr = 5)

If you look at TAnalyser.`Calculate_Residual_and_plot_TDC_Time_Diffs`, there is a magic number for slope and offset, which is curve fitted by the function below. feel free to change it after fitting a larger amount of data set

In [9]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser.plot_and_fit_tof_corrections()

0.15415730778671458 16.159064658832435
R² value: 0.6248481250576245


To look at each individual strip, one can use this function

In [10]:
importlib.reload(TTools)
TTools.show_strip_time_info(outDict, 20, 22, 2)

You can also plot the residual through this function

In [42]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser.plot_residual()

### Conclusion

The Timing analysis class contains all the functions needed to analyse the time walk effect, capturing timing corruptions in TDCs, and also monitoring tdcs as well. A general idea is that anything designed to run smoothly to produce the most useful result is written in TAnalyser with all instances stored internally. Analysis on individual bits will be in TTools. 

# Reconstructor
Very similar to the Timing Analyser, the Reconstructor's main goal is to reconstruct muon paths from the given data. The details of the code can be found under the documentation under the function RTools.`reconstruct_timed_Chi2_ByRPC`

below is an example of finding the efficiency of each RPC. The test RPC is removed, and a line is reconstructed using other 5 RPCs. This line is then extrapolated to the test RPCs' location, and a area of certain radius called the `tolerance` is probed for the hits. the success and failure together with their location is recorded

In [13]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        #Zone of Reconstruction
        if event_chunk:
            # We need to update the event like TAnalyser as well
            reconstructor.update_event(event_chunk, processedEvents)
            # populate_hits turns TDC bit wise information into their corresponding strips
            reconstructor.populate_hits()
            # This is obtionnal, and requires the residual of eta and phi
            reconstructor.apply_systematic_correction(residEta, residPhi)
            # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
            cluster = reconstructor.make_cluster()
            # Finally, recontruction is done using cluster information
            reconstructor.reconstruct_and_extrapolate(cluster)
        pbar.update(1)

Processing Events: 100%|██████████| 100/100 [00:32<00:00,  3.09Events/s]


Plotting the efficiency

In [15]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for RPC in range(6):
    if reconstructor.possible_reconstructions[RPC] == 0:
        efficiency = [0 for x in reconstructor.successful_reconstructions[RPC]]
    else:
        efficiency = [x / reconstructor.possible_reconstructions[RPC] for x in reconstructor.successful_reconstructions[RPC]]
    plt.plot(reconstructor.tol, efficiency, label=f'RPC {RPC}')

plt.xlabel('Tolerance')
plt.ylabel('Efficiency')
plt.title('idc what the title is')
plt.legend()
plt.grid(True)
plt.show()
print(reconstructor.possible_reconstructions)

[956, 1041, 1305, 1228, 1609, 1100]


You can also plot a heat map using the information obtained from the reconstruction

In [16]:
import matplotlib.colors as colors
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
rpcNames = {0: "Triplet Low", 1: "Triplet Mid", 2: "Triplet Top", 3: "Singlet", 4: "Doublet Low", 5: "Doublet Top"}

success_events = [[0 for etchan in range(32)] for phchan in range(64)]

with PdfPages('Data_output/reconstruction_heatmap_plots.pdf') as pdf:
    for rpc in range(6):
        for ph in range(64):
            for et in range(32):
                if reconstructor.successful_reconstructed_coords[rpc][ph][et] > 0:
                    total_successful = reconstructor.successful_reconstructed_coords[rpc][ph][et]
                    total_events = reconstructor.possible_reconstructions_coords[rpc][ph][et]
                    if total_events > 0:
                        success_events[ph][et] = total_successful / total_events
                    else:
                        success_events[ph][et] = 0  # No events, efficiency is 0

        fig, ax = plt.subplots(1, figsize=(16, 8), dpi=100)
        etachannels = [x - 0.5 for x in range(33)]
        phichannels = [x - 0.5 for x in range(65)]
        etaHist = (success_events, np.array(phichannels), np.array(etachannels))
        zrange = [0, max(max(row) for row in success_events)]
        thisHist = hep.hist2dplot(etaHist, norm=colors.Normalize(zrange[0], zrange[1]))
        thisHist.cbar.set_label('Successful reconstructions / Possible reconstructions', rotation=270, y=0.3, labelpad=23)
        plt.ylim(31.5, -0.5)
        plt.ylabel("Eta Channel")
        plt.xlabel("Phi Channel")
        ax.set_title(rpcNames[rpc])

        # Draw lines
        x_points = [-0.5, 64.5]
        y_points = [7.5, 15.5, 23.5]
        for y_point in y_points:
            plt.plot(x_points, [y_point, y_point], 'k', linestyle='dotted')
        y_points = [-0.5, 31.5]
        x_points = [7.5, 15.5, 23.5, 31.5, 39.5, 47.5, 55.5]
        for x_point in x_points:
            plt.plot([x_point, x_point], y_points, 'k', linestyle='dashed')

        pdf.savefig(fig)
        plt.close(fig)

print("PDF created successfully.")

PDF created successfully.


### Extracting angle information

One side product of reconstruction algorithm is the extraction of angles. This uses all 6 RPCs to reconstruct tracks, and find their angular distribution. The reason why a separate function is used to the efficiency calculation will be made clear later in the detailed explaination section

In [40]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents, tof_correction=True)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            # if processedEvents < 250:
            #     pbar.update(1)
            #     continue
            reconstructor.populate_hits()
            reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            filtered_events = RTools.filter_events(cluster,1,6)     
            reconstructor.extract_angles_phi_eta_timed_DZ_modified(filtered_events)
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:22<00:00,  6.68Events/s]


I was about to make this into a function, but then i need to write documentation for a code purely for plotting, might as well write it out here

In [41]:
import matplotlib.pyplot as plt
import numpy as np
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 20))
bin_edges = np.arange(-90.5, 91.5, 1)
phi_edges = np.arange(-180.5, 181.5, 1)
ax1.bar(bin_edges[:-1], reconstructor.eta_histogram, width=1, edgecolor='black', align='edge')
ax1.set_title('eta Angles Histogram chunk3')
ax1.set_xlabel('eta Angle (degrees)')
ax1.set_ylabel('Counts')

ax2.bar(bin_edges[:-1], reconstructor.phi_histogram, width=1, edgecolor='black', align='edge')
ax2.set_title('phi Angles Histogram chunk3')
ax2.set_xlabel('phi Angle (degrees)')
ax2.set_ylabel('Counts')

ax3.bar(phi_edges[:-1], reconstructor.solid_theta_histogram, width=1, edgecolor='black', align='edge')
ax3.set_title('solid theta Angles Histogram chunk3')
ax3.set_xlabel('solid theta Angle (degrees)')
ax3.set_ylabel('Counts')

ax4.bar(phi_edges[:-1], reconstructor.solid_phi_histogram, width=1, edgecolor='black', align='edge')
ax4.set_title('solid phi Angles Histogram chunk3')
ax4.set_xlabel('solid phi Angle (degrees)')
ax4.set_ylabel('Counts')


plt.tight_layout()
plt.show()

## Time of Flight analysis

Now we can do the recontruction and find the time of flight information across RPCs to determine our time resolution and also potential errors. This is another function called `reconstruct_and_findtof`, which records the hit time difference of the reconstructed paths by specifying which set of RPC one needs to compare

In [42]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
rpc_comparison = [[0,1], [0,2], [0,3], [0,4], [0,5]]
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents, tof_correction=True)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            # if processedEvents < 250:
            #     pbar.update(1)
            #     continue
            reconstructor.populate_hits()
            reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            reconstructor.reconstruct_and_findtof(cluster, rpc_comparisons=rpc_comparison)
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:28<00:00,  5.32Events/s]


In [43]:
importlib.reload(RTools)
RTools.compile_and_plot_tof(reconstructor.dT,rpc_comparison, pdf_filename='Data_output/tof.pdf')

Mid average value for RPC0-5: 1.3987860461696622
Gaussian fit parameters for RPC[0, 1]: amplitude = 53.50691046124199, mean = 1.2048419127332568, std deviation = 0.9790481241399833
Mid average value for RPC1-5: 0.8784803476790051
Gaussian fit parameters for RPC[0, 2]: amplitude = 53.7484141009169, mean = 0.6984959538885683, std deviation = -0.9981717275185302
Mid average value for RPC2-5: 2.6366421505285005
Gaussian fit parameters for RPC[0, 3]: amplitude = 52.43693916600671, mean = 2.357014405626686, std deviation = 0.8905643701367584
Mid average value for RPC3-5: 4.211738729375389
Gaussian fit parameters for RPC[0, 4]: amplitude = 43.36090025141649, mean = 5.095888569337571, std deviation = 1.0992980633930405
Mid average value for RPC4-5: 4.84941114066617
Gaussian fit parameters for RPC[0, 5]: amplitude = 44.0236360382068, mean = 5.349853481709834, std deviation = 1.0639506859991157


'Data_output/tof.pdf'

In [44]:
importlib.reload(RTools)
RTools.compile_and_plot_tof_chunk(reconstructor.dT,rpc_comparison, 10, pdf_filename='Data_output/tof_chunks.pdf')

Mid average value for RPC[0, 1], chunk1: 1.024887145104885
Gaussian fit parameters for RPC0-1: amplitude = 6.058657696836265, mean = 0.994628386001923, std deviation = -0.8725887113917731
Mid average value for RPC[0, 1], chunk2: 1.396897318093764
Gaussian fit parameters for RPC0-2: amplitude = 8.458237150315759, mean = 1.1614738324206986, std deviation = -0.4922944291647427
Mid average value for RPC[0, 1], chunk3: 1.346201154458408
Gaussian fit parameters for RPC0-3: amplitude = 4.530939846704952, mean = 1.341655804055145, std deviation = 1.1572258849713841
Mid average value for RPC[0, 1], chunk4: 1.2870266563429373
Gaussian fit parameters for RPC0-4: amplitude = 5.5814782660196975, mean = 1.4801366233995297, std deviation = -0.9433402364208442
Mid average value for RPC[0, 1], chunk5: 1.366932843589334
Gaussian fit parameters for RPC0-5: amplitude = 5.657049969015328, mean = 1.253895360273652, std deviation = 0.9672800854498189
Mid average value for RPC[0, 1], chunk6: 1.560018879318188

'Data_output/tof_chunks.pdf'

In [45]:
importlib.reload(proAnubis_Analysis_Tools)
reconstructor.plot_tof_offset(rpc_comparison)

c:\Users\jony\Programming\Python\Anubis\summer2\Tutorial with Osiris\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jony\Programming\Python\Anubis\summer2\Tutorial with Osiris\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
